In [4]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
import os
import pinecone
from tqdm.autonotebook import tqdm

In [2]:

os.environ["COHERE_API_KEY"] = "hv5YTaV6oUo5T9LGOY8F4bBtalGflhTU2FdPtEk3"
# initailize pinecode 

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

In [5]:
# load our document here 
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./BankFAQs.csv", encoding='utf8')

data = loader.load()

In [6]:
# split texts into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap  = 200,
    length_function = len,
)

data_chunks = text_splitter.split_documents(data)

In [7]:
# initialize embeddings 
embeddings = CohereEmbeddings(model = "multilingual-22-12")

In [9]:
# vector storage

db = Qdrant.from_documents(data_chunks, embeddings, location=":memory:", collection_name="my_documents", distance_func="Cosine")

In [37]:
# index_name = "faqs-dbase"

# docsearch = Pinecone.from_documents(data_chunks, embeddings, index_name=index_name)

In [40]:
# make our prompt 
prompt_template = """

generate response to the question based on the text provided.

Change instances where the bank is HDFC Bank to GTbank, 

If the text doesn't contain the answer, reply that the answer is not available and can request for more assistance by dialing the hotline or sending a mail to customer service represntative.

the hotline is 0300104523 and the mail is customer@gtbank.com.gh

Text: {context}

Question: {question}
"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [36]:
question = "how do i create a bank account"

In [41]:
# question and answer here 
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

answer = qa({"query": question})

In [42]:
print(answer['result'])


 To apply for a Super Saver Account with GTbank, you will need to visit a local branch with the following documents:

- A valid form of identification (ID)
- A recent utility bill or other proof of address
- Your Social Security number (SSN)

Once you have gathered the necessary documents, you can fill out an application form and submit it to a GTbank representative. They will help you through the process and answer any questions you may have.

If you have any further questions or need assistance with the application process, you can contact GTbank customer service by dialing 0300104523 or sending an email to customer@gtbank.com.gh.


In [32]:
def question_and_answer(question):
    qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

    results = qa({"query": question})

    answer = results['result']

    return answer

In [43]:
print(question_and_answer("How do  I request for visa card"))

 To request for a visa card with GTbank, you can visit the nearest GTbank branch with the required documents. You can also contact the customer service hotline or send an email to customer@gtbank.com.gh for more information. Please note that the requirements may vary based on your individual circumstances, so it's best to contact the bank directly to ensure you have the most up-to-date information.
